# BTC Alpha Lab — Bitcoin Alpha Discovery (Overnight)

This notebook documents a robust walk-forward backtest across **three strategy families**:

1. Trend-following (SMA crossover)
2. Mean-reversion (RSI threshold)
3. Volatility breakout/regime (Donchian breakout + EMA filter)

## Methodology
- Data ingestion from Stooq BTCUSD daily OHLC data
- Feature engineering: returns, SMAs, EMA regime, RSI, rolling highs/lows
- Backtesting with realistic assumptions:
  - signal lag of 1 day (avoid lookahead)
  - transaction costs applied on turnover
  - long/flat positioning
- Out-of-sample validation via walk-forward optimization:
  - train 3 years, test 1 year, rolling yearly
- Risk metrics: CAGR, Sharpe, Max Drawdown, Win Rate, annualized vol
- Sensitivity checks: cost stress test (2/5/10/20 bps)

⚠️ Educational research only. No guaranteed profits.


In [ ]:
import json
from pathlib import Path

root = Path('..').resolve()
summary = json.loads((root / 'results' / 'summary.json').read_text())
summary['meta']


## OOS Results Snapshot

- **vol_breakout**: CAGR 78.09% | Sharpe 1.33 | MaxDD -70.62% | WinRate 25.82%
- **trend_following**: CAGR 27.81% | Sharpe 0.74 | MaxDD -69.50% | WinRate 22.33%
- **mean_reversion**: CAGR 3.99% | Sharpe 0.29 | MaxDD -51.66% | WinRate 8.22%

### Best strategy in this test: **vol_breakout**

In [ ]:
# Pretty-print metric table without external dependencies
for name, m in sorted(summary['metrics_oos'].items(), key=lambda kv: kv[1]['Sharpe'], reverse=True):
    print(f"{name:15s}  CAGR={m['CAGR']:.2%}  Sharpe={m['Sharpe']:.2f}  MaxDD={m['Max Drawdown']:.2%}  WinRate={m['Win Rate']:.2%}")


In [ ]:
# Cost sensitivity view (Sharpe by transaction cost)
for strat, vals in summary['cost_sensitivity'].items():
    print(f"\n{strat}")
    for row in vals:
        print(f"  cost={row['cost_bps']:>2} bps -> Sharpe={row['Sharpe']:.2f}, CAGR={row['CAGR']:.2%}, MaxDD={row['MaxDD']:.2%}")


## Recommendation

Based on this run, the strongest out-of-sample Sharpe came from **vol_breakout**.

Practical deployment plan:
1. Paper-trade first for 4-8 weeks with fixed parameters.
2. Add portfolio-level risk limits (e.g., 20% strategy drawdown cut-off).
3. Include slippage + exchange fee schedule per venue before live trading.
4. Re-train only on a fixed quarterly schedule to reduce overfitting.
5. Monitor live-vs-backtest drift and disable strategy if Sharpe collapses.

### Caveats
- Crypto regime shifts are frequent; past edge may decay.
- Daily bars ignore intraday path/risk.
- Max drawdown is high; leverage would be dangerous.
- **No guaranteed profits.**


In [ ]:
# Reproducibility: rerun full research pipeline
# !python3 ../research.py
